<a href="https://colab.research.google.com/github/Shrimpen/Ai-projekt/blob/master/Create_SuperVillain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Skapa en konstverks AI med GAN
I denna Colab ska vi med hjälp av GAN (Generative adversarial network) skapa konst, mikrohjältar, pokemonkort eller vad man nu vill!
I guiden nedan av Nathan Glover kan man få lite ytterligare tips och inspiration. Det är även denna guides kod som vi använder som grund.
[How to generate pokemon cards](https://devopstar.com/2019/05/21/stylegan-pokemon-card-generator/)

I koden kör man en algoritm som heter Stylegan, det finns sedan slutet av december [Stylegan 2](https://github.com/NVlabs/stylegan2), den kan man också testa på att använda om man vill.



# Förarbete
Börja med att clona github repon. Sen ska du köra cellen nedan för att hämta in den kod vi behöver och att ha rätt version av tensorflow.


In [3]:
%tensorflow_version 1.x

%cd /content/
!git clone https://github.com/t04glovern/stylegan-pokemon
%cd /content/stylegan-pokemon/
!mkdir Weights
%cd /content/stylegan-pokemon/Weights
!wget https://s3.amazonaws.com/devopstar/resources/stylegan-pokemon/MichaelFriese10_pokemon.pkl


TensorFlow 1.x selected.
/content
fatal: destination path 'stylegan-pokemon' already exists and is not an empty directory.
/content/stylegan-pokemon
mkdir: cannot create directory ‘Weights’: File exists
/content/stylegan-pokemon/Weights
--2020-04-09 13:01:13--  https://s3.amazonaws.com/devopstar/resources/stylegan-pokemon/MichaelFriese10_pokemon.pkl
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.97.211
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.97.211|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 302550530 (289M) [application/x-www-form-urlencoded]
Saving to: ‘MichaelFriese10_pokemon.pkl.1’

MichaelFriese10_pok 100%[===================>] 288.53M  46.1MB/s    in 6.7s    

2020-04-09 13:01:20 (42.9 MB/s) - ‘MichaelFriese10_pokemon.pkl.1’ saved [302550530/302550530]



#Spara och ladda in från Google Drive
Vi börjar med att ansluta till Google drive och skapa mappar för STYLE-GAN om vi inte redan har det. Vi skapar en länk till den nyskapade mappen som vi kallar STYLE-GAN och kommer använda här i koden för att enklare komma åt filer från mappen.
### Mer detaljerat:
Du börjar med att importera drive, när du på nästa rad mountar den(Ger dig själv åtkomst) får du in på en länk, kopiera en kod och mata in den i terminalen under kodfönstret. Efter det stegar vi in i våran drive, skapar mappen ai och läser vilka mappar som finns där för att se om den skapades.

Efter det går vi tillbaka till content igen(Våran Colab files mapp) och lägger en länkad genväg till ai mappen som vi bara kallar /ai, då kommer vi komma åt ai mappens undermappar och filer direkt i colab genom att bara skriva /ai.

In [4]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount("/content/drive", force_remount=True)
%cd /content/drive/My Drive/
!mkdir STYLE-GAN
%ls
%cd /content

!ln -s "/content/drive/My Drive/STYLE-GAN" /STYLE-GAN
%cd /STYLE-GAN
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive
mkdir: cannot create directory ‘STYLE-GAN’: File exists
STYLE-GAN/
/content
/content/drive/My Drive/STYLE-GAN
STYLE-GAN@


För att ladda in data från driven så använder vi våra snabblänk /STYLE-GAN. Se i nästa kapitlet


#Ladda in förtränad data och vikt för att fortsätta träna



När man tränar/ fortsätter träna så måste man ha en mapp med pkl filen. Det vill säga vikten och en mapp med .tfrecords det vill säga datat.

Nu när vi har synkat till driven via länken /STYLE-GAN så kommer vi använda den för att komma åt data och möjliga viktfilen. Vi måste gå in tre filer i mappen stylegan (/content/stylegan-pokemon/stylegan).
Dessa är config.py, training_loop.py och train.py.

Config filen som man hör har koll på vilka platser man ska ladda in och spara till ifrån. 

U train säger man vart datat ligger och hur stor datafilen är.

I training_loop berättar vi vilken viktfil vi ska köra och på vilket "varv" den ska starta på. Varvet är där den pkl filen slutade på och när man  har tränat ett "tick" så sparas en pkl fil t.ex `network-snapshot-006085.pkl` då vet man att varvet(resume_kimg) är 6085.





## Dessa ska ändras för att det ska fungera.
### Training_loop:  
#### Fortsätta på påbörjad träning
Hitta och ändra till dessa två rader för att börja med en redan påbörjad viktfil.
Den första raden säger vart filen ligger och den andra vilken start kimg värde den har.
`       resume_run_id           = "/STYLE-GAN/start.pkl",     # Run ID or network pkl to resume training from, None = start from scratch. `

`    resume_kimg             = 6525, `
#### Påbörja ny träning
Vill man börja från början ändrar man dem till detta:
`       resume_run_id           = None,     # Run ID or network pkl to resume training from, None = start from scratch. `

`    resume_kimg             = 0, `



### train.py 
Ändra namnet där datan ligger, det vill säga tffilerna. Vi vill också ändra upplösningen till den vi valt att bilderna ska ha (i mitt fall 256*256 pixlar).
```
    desc += '-custom';     dataset = EasyDict(tfrecord_dir='data', resolution=256);  
```
### Config.py
Ändrar sökvägen för vart vår data ligger och vart resultatet ska sparas.
```
result_dir = '/STYLE-GAN/results'
data_dir = '/STYLE-GAN'

```



Efter dessa ändringar är det bara att köra train.py filen.



In [8]:
#Train
%cd /content/stylegan-pokemon/stylegan
!python train.py

/content/stylegan-pokemon/stylegan



Creating the run dir root: /STYLE-GAN/results
Creating the run dir: /STYLE-GAN/results/00000-sgan-custom-1gpu
Copying files to the run dir
dnnlib: Running training.training_loop.training_loop() on localhost...



Streaming data using training.dataset.TFRecordDataset...


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_classes(iterator)`.
Dataset shape = [3, 256, 256]
Dynamic range = [0, 255]
Label size    = 0
Constructing networks...







G  

Såhär kan det se ut när man tränar:
```
tick 1     kimg 6205.0   lod 0.00  minibatch 8    time 1h 09m 09s   sec/tick 4082.9  sec/kimg 102.07  maintenance 65.6   gpumem 4.9 
tick 2     kimg 6245.0   lod 0.00  minibatch 8    time 2h 17m 19s   sec/tick 4074.3  sec/kimg 101.86  maintenance 15.8   gpumem 4.9 
tick 3     kimg 6285.0   lod 0.00  minibatch 8    time 3h 25m 21s   sec/tick 4076.6  sec/kimg 101.91  maintenance 6.0    gpumem 4.9 
tick 4     kimg 6325.0   lod 0.00  minibatch 8    time 4h 33m 23s   sec/tick 4075.4  sec/kimg 101.89  maintenance 6.2    gpumem 4.9 
tick 5     kimg 6365.0   lod 0.00  minibatch 8    time 5h 41m 18s   sec/tick 4069.3  sec/kimg 101.73  maintenance 5.9    gpumem 4.9 
tick 6     kimg 6405.0   lod 0.00  minibatch 8    time 6h 49m 14s   sec/tick 4069.8  sec/kimg 101.75  maintenance 5.7    gpumem 4.9 
tick 7     kimg 6445.0   lod 0.00  minibatch 8    time 7h 57m 12s   sec/tick 4072.4  sec/kimg 101.81  maintenance 5.7    gpumem 4.9 
tick 8     kimg 6485.0   lod 0.00  minibatch 8    time 9h 05m 13s   sec/tick 4075.1  sec/kimg 101.88  maintenance 5.8    gpumem 4.9
```
Det ni behöver veta om det är att den första kolumnen berättar hur många tick har gått, den andra hur mycket bättre AIn blir på att rita av träningsdatat. Flkht av detta är hur många batcher det vill säga delar den delar upp det i och tisvariabler som hur många sekunder detta tick tog och den totala körtiden.

När skriptet körs skapas en ny mapp i /STYLE-GAN/results som heter något i stil med **00004-sgan-custom-1gpu**, där kommer fakebilder och vikter för varje tick sparas. På fakebilderna ser man hur duktig AIn är på att rita likt den riktiga "konstnären".  Vikterna kan du sedan välja att använda för att träna vidare eller skapa konstverk utav.

##TIPS
När jag kört klart en träningsrunda (8-10 timmar) så tar jag den nyaste vikten och lägger i /STYLE-GAN mappen och byter namn på den till start.pkl. Då får man kolla på fakebilden för att se vilket varv(kimg) den var på och så använder man den som start i nästa träningsrunda.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Samla in och skapa dataset


1.   Skapa mapparna raw och datapng. I raw kommer du lägga de nedladdade bilderna, vi kommer senare konvertera dem till png och flytta över dem till datapng.
2.   Nu ska du hämta vart ifrån du vill ha dina bilder som ska bli ett data sätt. Du kan antingen ladda in egna bilder genom att dra dom till mappen datapng. Eller så kan du köra !wget på den sidan där det finns bilderna du vill hämta. Wgetn säger att vi vill spara bilderna i raw, vi vill låta den ladda ner alla bildtyper som är jpeg,jpg,png och gif från den angivnla länken.



In [0]:
!wget  -nd -r -P  /content/stylegan-pokemon/stylegan/Microheroes/raw -A jpeg,jpg,png,gif http://houseofmicros.x10host.com/images/Marvel/Minis/FairyTales/




3.   Nu går vi in i stylegan och kör prepare.py om vi vill använda den förinstallerade prepare koden(Kika gärna på den). Den gör om alla bilder till samma storlek (512x512). Detta kan man ändra till andra tal i den numeriska talföljden (ex 124, 256 osv). Det den även gör är att den drar ut bilderna, detta fungerar bra i vissa fall men sämre i andra.
Av den anledningen så har jag gjort en egen prepare nedan som man kan köra om man istället vill behålla bildens egentliga storlek och bara ändra dess yta.
4.   Efter det kör vi dataset_tool.py som gör om bilderna till ett dataset. Jag har lagt att den ska sparas i data. Då skapas en map som heter data med dit dataset. Flytta sedan in den i din STYLE-GAN mapp för att få upp den till din drive
5.   Nu är du klar med dit dataset och kan gå upp till "Ladda in förtränad data och vikt för att fortsätta träna" och fortsätta med stegen därifrån. 


In [5]:
#Prepare code
from PIL import Image
import os, sys

path ="/raw/"
out="/data1/"

dirs = os.listdir( path )
n=0
count=0
for item in dirs:
          try:
              if os.path.isfile(path+item):
                  im = Image.open(path+item)
                  
                  background = Image.new('RGB', (256, 256), (255, 255,255))
                  #background.paste(im, mask=im.split()[3]) #För Gif (transparenta)
                  #background.paste(im, mask=im.split()[3]) #För Bilder med bakgrundsfärg
                  background.save(out +  item +'.png', 'png', quality=90)
                  n+=1
              
          except Exception as e:
              print(e)
print("Done with converting ",n, "to png.")

Done with converting  569 to png.


In [7]:
%cd /content/stylegan-pokemon/stylegan

#!python prepare.py
!python dataset_tool.py create_from_images /data /datapng

/content/stylegan-pokemon/stylegan



Loading images from "/data1"
Creating dataset "/STYLE_GAN/data"



Added 569 images.


### Ta bort bildfilerna.
Om man vill ta bort filerna så kör följande kommandon, annars försvinner de ändå automatiskt när runtimen är över.

In [0]:
# !rm /content/stylegan-pokemon/stylegan/Microheroes/data/*
# !rm /content/stylegan-pokemon/stylegan/Microheroes/raw/*

StyleGan 2
[stylegan2](https://github.com/NVlabs/stylegan2)